In [354]:
import pandas as pd
import json
import numpy as np

with open("declarations.json", "r") as f:
    data = json.load(f)
    
df = pd.DataFrame(data)

gosduma_df = df[ df.main.apply(lambda x: x['office']['name']) == "Государственная Дума"]
gosduma_df.head()

main  \
0  {'person': {'id': 8, 'name': 'Зюганов Геннадий...   
1  {'person': {'id': 9, 'name': 'Алферов Жорес Ив...   
2  {'person': {'id': 10, 'name': 'Андреев Андрей ...   
3  {'person': {'id': 11, 'name': 'Апарина Алевтин...   
4  {'person': {'id': 12, 'name': 'Афонин Юрий Вяч...   

                                             incomes  \
0  [{'size': 2126487.0, 'relative': None, 'commen...   
1  [{'size': 8338604.93, 'relative': None, 'comme...   
2  [{'size': 2078056.0, 'relative': None, 'commen...   
3  [{'size': 2133612.0, 'relative': None, 'commen...   
4  [{'size': 11192197.0, 'relative': None, 'comme...   

                                        real_estates  \
0  [{'name': '', 'square': 167.4, 'country': 'Рос...   
1  [{'name': '', 'square': 5000.0, 'country': 'Ро...   
2                                                 []   
3  [{'name': '', 'square': 56.0, 'country': 'Росс...   
4  [{'name': '', 'square': 1694.0, 'country': 'Ро...   

                                            vehicles  \
0                                                 []   
1  [{'name': '', 'type': {'id': 1, 'name': 'Автом...   
2  [{'name': '', 'type': {'id': 1, 'name': 'Автом...   
3                                                 []   
4  [{'name': '2.0', 'type': {'id': 1, 'name': 'Ав...   

                                             savings  \
0  [2 652 390,0 руб. (ВТБ 24 (ЗАО)), 474 061,0 ру...   
1  [24 917 309,0 руб. (АКБ "Связь-Банк"), 10 039 ...   
2  [0,0 руб. (ОАО «Альфа-банк»), 17,0 руб. (ОАО А...   
3           [524 008,0 руб. (ОАО "Сбербанк России")]   
4  [78,0 руб. (ВТБ 24 (ОАО)), 88 951,0 руб. (ОАО ...   

                                              stocks bonds spendings  
0                                                 []    []        []  
1                                                 []    []        []  
2                                                 []    []        []  
3                                                 []    []        []  
4  [1100 шт. ОАО"Группа Илим", 1.0 доли, 5 шт. ОА...    []        []

In [2]:
with open("data/declarator_car_brands_map.json", "r") as f:
    car_brands_map = json.load(f)
car_brands_map = dict(zip([x['id'] for x in car_brands_map], car_brands_map))
car_brands_map[-1] = {}

In [3]:
(gosduma_df.vehicles.apply(len) > 0).mean()

0.7526306402710897

In [4]:
import itertools
vehicles = gosduma_df.vehicles[gosduma_df.vehicles.apply(len) > 0]
vehicles = list(itertools.chain(*vehicles))
vehicles_df = pd.DataFrame(vehicles)

In [5]:
vehicles_df['type_id'] = vehicles_df['type'].apply(lambda x: x['id'])
vehicles_df['type_name'] = vehicles_df['type'].apply(lambda x: x['name'])



vehicles_df['brand_name'] = vehicles_df['brand'].apply(lambda x: x['name'] if x is not None else None)
vehicles_df['brand_id'] = vehicles_df['brand'].apply(lambda x: x['id'] if x is not None else -1)

vehicles_df['parent_brand_name'] = [car_brands_map[x].get('parent_name') for x in vehicles_df.brand_id ]

vehicles = vehicles_df.drop(["brand", "type"], axis=1)

In [6]:
((vehicles.name == "") | (vehicles.name == " ")).mean()

0.33833913475882643

In [7]:
vehicle_names = (vehicles.parent_brand_name + " " + vehicles.name + " " + vehicles.brand_name.fillna("")).dropna()
vehicle_names = vehicle_names[vehicle_names != " "]
vehicle_names = vehicle_names.str.strip()

# Load data from multiple sources and unite it

In [8]:
cars_df = pd.read_csv("data/true_car_listings.csv")
cars_df.drop(["Mileage", "City", "State", "Vin"], axis =1 , inplace = True)
cars_df = cars_df.groupby(["Make", "Model", "Year"]).mean().reset_index()
cars_df.head() # Dollars. Convert by average $USD of 2017 Year
cars_df.columns = ["brand", "model", "year", "price"]
cars_df['price'] = cars_df.price *62.830985
cars_df.head()

car_base = cars_df

In [9]:
cars_df = pd.read_csv("data/data.csv")
cars_df = cars_df[["Make", "Model", "Year", "MSRP"]] # TODO in which currency?
cars_df = cars_df.groupby(["Make", "Model", "Year"]).mean().reset_index()
cars_df.columns = ["brand", "model", "year", "price"]
cars_df['price'] = cars_df.price *62.830985
cars_df.head()
cars_msrp = cars_df

In [56]:
from unidecode import unidecode

cars_df = pd.read_csv("data/autos.csv", sep=',', header=0, encoding='cp1252')
cars_df = cars_df[["name", "price", "yearOfRegistration", "model" , "brand"]]
cars_df.name =cars_df.name.str.replace('_', " ").fillna("").apply(unidecode)
cars_df.brand = cars_df.brand.str.replace("_", " ").fillna("").apply(unidecode)
cars_df.model = cars_df.model.str.replace("_", " ").fillna("").apply(unidecode)
cars_df.columns = ['name', "price", "year", "model", "brand"]
cars_df = cars_df.drop_duplicates()
cars_df['full_name'] = cars_df.name + ' ' + cars_df.brand + ' ' + cars_df.model
cars_df['price'] = cars_df.price * 71.8
# Remove punctiation

cars_df.head()

name      price  year  model       brand  \
0                      Golf 3 1.6    34464.0  1993   golf  volkswagen   
1            A5 Sportback 2.7 Tdi  1313940.0  2011               audi   
2  Jeep Grand Cherokee "Overland"   703640.0  2004  grand        jeep   
3              GOLF 4 1 4  3TURER   107700.0  2001   golf  volkswagen   
4  Skoda Fabia 1.4 TDI PD Classic   258480.0  2008  fabia       skoda   

                                    full_name  
0                  Golf 3 1.6 volkswagen golf  
1                  A5 Sportback 2.7 Tdi audi   
2   Jeep Grand Cherokee "Overland" jeep grand  
3          GOLF 4 1 4  3TURER volkswagen golf  
4  Skoda Fabia 1.4 TDI PD Classic skoda fabia

# Matching with dumps

In [113]:
from fuzzywuzzy import process, fuzz

example = vehicle_names[8]
example

'Hyundai 2.7 GLS AT Santa Fe'

In [107]:
choices = cars_df.full_name
results = process.extractBests(example, choices, score_cutoff=90, limit=10)
result_keys = list([r[2] for r in results])

In [120]:
prices = cars_df.loc[result_keys].sort_values(by="year")


In [122]:
prices

name     price  year  model    brand  \
53195   Hyundai Santa Fe  183449.0  2001  santa  hyundai   
571     Hyundai Santa Fe  344640.0  2004  santa  hyundai   
55446   Hyundai Santa Fe  179500.0  2004  santa  hyundai   
133430  Hyundai Santa Fe  201040.0  2004  santa  hyundai   
17166   Hyundai Santa Fe  312330.0  2005  santa  hyundai   
48984   Hyundai Santa Fe  524068.2  2006  santa  hyundai   
138508  Hyundai Santa FE  522704.0  2006  santa  hyundai   
145179  Hyundai Santa Fe  537782.0  2008  santa  hyundai   
20776   Hyundai santa fe  201040.0  2017  santa  hyundai   
105437  Hyundai santa fe  208220.0  2017  santa  hyundai   

                             full_name  
53195   Hyundai Santa Fe hyundai santa  
571     Hyundai Santa Fe hyundai santa  
55446   Hyundai Santa Fe hyundai santa  
133430  Hyundai Santa Fe hyundai santa  
17166   Hyundai Santa Fe hyundai santa  
48984   Hyundai Santa Fe hyundai santa  
138508  Hyundai Santa FE hyundai santa  
145179  Hyundai Santa Fe hyundai santa  
20776   Hyundai santa fe hyundai santa  
105437  Hyundai santa fe hyundai santa

# Plot

In [470]:
import scipy.stats as sts

def get_dots_from_prices(prices: np.array):
    kde = sts.gaussian_kde(prices)
    _std = prices.std()
    xs = np.linspace(prices.min() - _std.std()*3, prices.max() + _std*3, num=150)
    ys = kde.pdf(xs)
    return xs, ys

In [162]:
car_db = cars_df.copy()

choices = car_db.full_name

def get_relevant_car_prices(query, year=None, upcount=True):
    results = process.extractBests(query, choices, score_cutoff=90, limit=10)
    result_keys = list([r[2] for r in results])
    relevant_cp = car_db.loc[result_keys].sort_values(by="year")
    
    prices = relevant_cp.price
    
    if upcount and year:
        year_dist = (relevant_cp.year - year)
        prices = prices + (prices * (.015*year_dist));
        
    return prices, relevant_cp.year


In [254]:
cars_msrp.head()

brand model  year         price
0  Acura    CL  2001  1.820842e+06
1  Acura    CL  2002  1.823983e+06
2  Acura    CL  2003  1.914460e+06
3  Acura   ILX  2015  1.845660e+06
4  Acura   ILX  2016  1.954881e+06

In [266]:
choices_msrp = (cars_msrp.brand + " " + cars_msrp.model + " " + cars_msrp.year.astype("str"))
def get_msrp_price(query=None, brand=None, model=None, year=None):
    
    if query is None:
        target_brand = process.extractOne(brand, choices=cars_msrp.brand)
        subset = cars_msrp[cars_msrp.brand == target_brand[0]]
        print(subset, target_brand)
        target_model = process.extractOne(model, choices=subset.model)
        return subset.loc[target_model[2]]
    
    result = process.extractOne(query, choices=choices_msrp)
    if result[1] > 92:
        return cars_msrp.loc[result[2]]
    else:
        return None

In [267]:
choices_cardeal = (car_base.brand + " " + car_base.model + " " + car_base.year.astype("str"))
def get_cardeal_prices(query, year=None, upcount=True):
    results = process.extractBests(query, choices_cardeal, score_cutoff=90, limit=10)
    result_keys = list([r[2] for r in results])
    print(results)
    relevant_cp = car_base.loc[result_keys].sort_values(by="year")
    
    prices = relevant_cp.price
    
    if upcount and year:
        year_dist = (relevant_cp.year - year)
        prices = prices + (prices * (.015*year_dist));
        
    return prices, relevant_cp.year

# Integrate with declaration entities

In [268]:
ex = df.vehicles[132][0]
ex

{'name': 'GD 2243',
 'type': {'id': 1, 'name': 'Автомобиль легковой'},
 'manufacture_year': 2004,
 'brand': {'id': 863, 'name': 'Magentis'},
 'comment': '',
 'relative': None}

In [378]:
def get_vehicle_query(vehicle):
    vehicle = vehicle.copy()
    vehicle['type_id'] = vehicle['type']['id']
    vehicle['type_name'] = vehicle['type']['name']


    vehicle['brand_name'] = vehicle['brand']['name'] if vehicle['brand'] is not None else None
    vehicle['brand_id'] = vehicle['brand']['id'] if vehicle['brand'] is not None else -1

    vehicle['parent_brand_name'] = car_brands_map[vehicle['brand_id']].get('parent_name')
    
    if  vehicle['parent_brand_name'] is None:
        vehicle['parent_brand_name'] = ""
    
    if not vehicle['name']:
        vehicle['name'] = ""
    
    if not vehicle['brand_name']:
        vehicle['brand_name'] = ""
        
    vehicle_query = (vehicle['parent_brand_name'] + " " + vehicle['name'] + " " + vehicle['brand_name'])
    
    vehicle_query = vehicle_query.strip()
    
    year = vehicle['manufacture_year']
    
    return(vehicle_query, year)


In [379]:
def get_vehicle_brand_model_year(vehicle):
    vehicle = vehicle.copy()
    vehicle['type_id'] = vehicle['type']['id']
    vehicle['type_name'] = vehicle['type']['name']


    vehicle['brand_name'] = vehicle['brand']['name'] if vehicle['brand'] is not None else None
    vehicle['brand_id'] = vehicle['brand']['id'] if vehicle['brand'] is not None else -1

    vehicle['parent_brand_name'] = car_brands_map[vehicle['brand_id']].get('parent_name')
    
    if not vehicle['parent_brand_name']:
        vehicle['parent_brand_name'] = ""
    
    if not vehicle['brand_name']:
        vehicle['brand_name'] = ""
        
    if not vehicle['name']:
        vehicle['name'] = ""
    
    return vehicle['parent_brand_name'].strip(), (vehicle['brand_name'] + " " + vehicle['name']).strip(), vehicle['manufacture_year']


In [366]:
def get_prices(v):
    brand, model, year = get_vehicle_brand_model_year(v)
    if not any(lambda x: x == "" or x == " ", [brand, model]):
        # API and MSRP are available
        return get_msrp_price()

In [368]:
# brand, model, year = get_vehicle_brand_model_year(ex)
# query, year = get_vehicle_query(ex)

In [369]:
get_cardeal_prices(query, year)

[]


(Series([], dtype: float64), Series([], Name: year, dtype: int64))

In [286]:
get_relevant_car_prices(query, year)

(273425    284722.9
 241472     33889.6
 dtype: float64, 273425    2006
 241472    2016
 Name: year, dtype: int64)

In [288]:
import requests
import requests_cache

requests_cache.install_cache('test_cache', backend='sqlite', expire_after=150000)

In [394]:
def ria(brand, model, year):
    RIA_API_KEY = "X5dCPikYx8iNeKiWy9hBOy4oe4JXkkFuZ2a4hlzV"
    average_price_url = "https://developers.ria.com/auto/average_price"


    brands_url = "https://developers.ria.com/auto/categories/1/marks?"
    brands = requests.get(brands_url, params={"api_key": RIA_API_KEY,}).json()
    brands_dict = dict(zip([x['name'] for x in brands], [x['value'] for x in brands]))

    brand_name = brands_dict.get(brand)
    if brand_name:
        models_url = f"https://developers.ria.com/auto/categories/1/marks/{brand_name}/models?api_key={RIA_API_KEY}"
        brand_models = requests.get(models_url).json()
        model_names, model_ids = [x['name'] for x in brand_models], [x['value'] for x in brand_models]
        model_dict = dict(zip(model_names, model_ids))
        model_result = process.extractOne(model, choices=model_names)
        model_name = model_dict[model_result[0]]
    
        if model_name:
            result = requests.get(average_price_url, params={"api_key": RIA_API_KEY,
                                            "marka_id": brand_name, 
                                            "model_id": model_name,
                                            "years": year})
            if result.status_code == 200:
                result = result.json()['prices']
                result = np.array(result) * 64
                return result
            elif result.status_code == 429:
                raise Exception("Too many reqs")
            else:
                return None
        else:
            return None
    else:
        return None

In [326]:
ria("Lada", "Granta", 2018)

array([635794.7904, 332800.    , 512000.    , 629066.8032, 637476.7872,
       639668.48  , 288000.    , 268736.    , 371200.    , 326400.    ,
       320000.    , 345600.    , 153600.    , 627767.0784, 320000.    ,
       352000.    , 356787.2   , 304000.    ])

# PreCompute

In [413]:
import itertools
gosduma_w_v = gosduma_df[gosduma_df.vehicles.apply(len) > 0]

In [414]:
all_vehicles = []
for row in gosduma_w_v.itertuples():
    vs = row[4]
    for i, _v in enumerate(vs):
        _v['declaration_id'] = row.Index
        _v['vehicle_num'] = i

        _v['query'] = get_vehicle_query(_v)
        brand, model, year = get_vehicle_brand_model_year(_v)
        _v['_brand'] = brand
        _v['_model'] = model
        _v["_year"] = year
        
    all_vehicles.extend(vs)

In [415]:
all_df = pd.DataFrame(all_vehicles)
ok_brand = all_df._brand != ""
ok_model = all_df._model != ""

1

In [466]:
from tqdm.auto import tqdm
from time import sleep

ria_respones = []
for row in tqdm(list(all_df[ok_brand & ok_model].itertuples())[3442:]):
    response = {"declaration_id": row.declaration_id, "vehicle_num": row.vehicle_num}
    if row.type['id'] == 1:   
        r = ria(row._9, row._10, row._11)
        response["response"] = r
        ria_respones.append(response)
        sleep(0.1)


In [467]:
ria_respones = np.array(ria_respones)

In [433]:
np.save("ria_responses_part_2",  ria_respones)

In [468]:
np.mean([r['response'] is None for r in ria_respones])

0.004456633527596846

In [439]:
ria_respones.shape

(3442,)

In [441]:
all_df[ok_brand & ok_model].shape

(6362, 12)

In [445]:
gosduma_w_v['declaration_id'] = gosduma_w_v.index

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [458]:
gosduma_w_v.to_csv("gosduma_w_vehicles.csv", index=False)

In [469]:
import pickle
with open("ria_responses_part_2.pickle", "bw+") as f:
    pickle.dump(ria_respones, f)

In [457]:
for i, r in gosduma_w_v.iterrows():
    collection.insertOne(dict(r))
    break

{'main': {'person': {'id': 9, 'name': 'Алферов Жорес Иванович', 'family_name': 'Алферов', 'given_name': 'Жорес', 'patronymic_name': 'Иванович', 'gender': 'M'}, 'office': {'id': 14, 'name': 'Государственная Дума', 'url': 'http://www.duma.gov.ru/', 'type': {'id': 7, 'name': 'Федеральный, без региональной структуры'}, 'region': None}, 'party': {'id': 6, 'name': 'КПРФ'}, 'year': 2010, 'document_type': {'id': 2, 'name': 'Предвыборная декларация'}}, 'incomes': [{'size': 8338604.93, 'relative': None, 'comment': 'Университет Конкук,  пенсия, прочее,  Российская академия наук,  Государственное образовательное учреждение высшего профессионального образования "Санкт-Петербургский Политехнический университет",  аппарат Государственной Думы Федерального Собрания Российской Федерации,  Учреждение Российской академии наук Санкт-Петербургский научный центр РАН'}], 'real_estates': [{'name': '', 'square': 5000.0, 'country': 'Россия', 'region': {'id': 1, 'name': 'Санкт-Петербург'}, 'comment': '', 'type':

In [459]:
pd.read_csv("gosduma_w_vehicles.csv")

main  \
0     {'person': {'id': 9, 'name': 'Алферов Жорес Ив...   
1     {'person': {'id': 10, 'name': 'Андреев Андрей ...   
2     {'person': {'id': 12, 'name': 'Афонин Юрий Вяч...   
3     {'person': {'id': 18, 'name': 'Езерский Никола...   
4     {'person': {'id': 19, 'name': 'Калашников Леон...   
...                                                 ...   
4215  {'person': {'id': 98, 'name': 'Напсо Юрий Аисо...   
4216  {'person': {'id': 162, 'name': 'Резник Владисл...   
4217  {'person': {'id': 55, 'name': 'Савицкая Светла...   
4218  {'person': {'id': 14248, 'name': 'Серпер Евген...   
4219  {'person': {'id': 131678, 'name': 'Дивейкин Иг...   

                                                incomes  \
0     [{'size': 8338604.93, 'relative': None, 'comme...   
1     [{'size': 2078056.0, 'relative': None, 'commen...   
2     [{'size': 11192197.0, 'relative': None, 'comme...   
3     [{'size': 7717428.05, 'relative': None, 'comme...   
4     [{'size': 1466358.09, 'relative': None, 'comme...   
...                                                 ...   
4215  [{'size': 4628193.24, 'relative': None, 'comme...   
4216  [{'size': 29773034.37, 'relative': None, 'comm...   
4217  [{'size': 14760204.85, 'relative': None, 'comm...   
4218  [{'size': 7828038.55, 'relative': None, 'comme...   
4219  [{'size': 6049426.82, 'relative': None, 'comme...   

                                           real_estates  \
0     [{'name': '', 'square': 5000.0, 'country': 'Ро...   
1                                                    []   
2     [{'name': '', 'square': 1694.0, 'country': 'Ро...   
3                                                    []   
4     [{'name': '', 'square': 1520.0, 'country': 'Ро...   
...                                                 ...   
4215  [{'name': '', 'square': 66.6, 'country': 'Росс...   
4216  [{'name': '', 'square': 22500.0, 'country': 'Р...   
4217  [{'name': '', 'square': 207.0, 'country': 'Рос...   
4218  [{'name': '', 'square': 27.2, 'country': 'Росс...   
4219  [{'name': '', 'square': 131.3, 'country': 'Рос...   

                                               vehicles  \
0     [{'name': '', 'type': {'id': 1, 'name': 'Автом...   
1     [{'name': '', 'type': {'id': 1, 'name': 'Автом...   
2     [{'name': '2.0', 'type': {'id': 1, 'name': 'Ав...   
3     [{'name': 'S500', 'type': {'id': 1, 'name': 'А...   
4     [{'name': '21053', 'type': {'id': 1, 'name': '...   
...                                                 ...   
4215  [{'name': 'Мотолодка Princercaft-Vogve 27SE', ...   
4216  [{'name': '', 'type': {'id': 1, 'name': 'Автом...   
4217  [{'name': '', 'type': {'id': 1, 'name': 'Автом...   
4218  [{'name': 'Wanderer', 'type': {'id': 1, 'name'...   
4219  [{'name': 'BMW X5 XDRIVE40D', 'type': {'id': 1...   

                                                savings  \
0     ['24 917 309,0 руб. (АКБ "Связь-Банк")', '10 0...   
1     ['0,0 руб. (ОАО «Альфа-банк»)', '17,0 руб. (ОА...   
2     ['78,0 руб. (ВТБ 24 (ОАО))', '88 951,0 руб. (О...   
3           ['28 685 623,0 руб. (ОАО Сбербанк России)']   
4                 ['36 598,0 руб. (ОАО АКБ "РОСБАНК")']   
...                                                 ...   
4215                                                 []   
4216                                                 []   
4217                                                 []   
4218                                                 []   
4219                                                 []   

                                                 stocks bonds spendings  \
0                                                    []    []        []   
1                                                    []    []        []   
2     ['1100 шт. ОАО"Группа Илим", 1.0 доли', '5 шт....    []        []   
3                                                    []    []        []   
4                                                    []    []        []   
...                                                 ... 